In [1]:
!pip install --upgrade datasets
!pip install --upgrade huggingface_hub
!pip install patool

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.4/436.4 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingfa

In [2]:
from huggingface_hub import notebook_login
from datasets import load_dataset
notebook_login()

In [3]:
from urllib.request import urlretrieve

url = (
    "https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar"
)

filename = "ILSVRC2012_img_val.tar"

urlretrieve(url, filename)

('ILSVRC2012_img_val.tar', <http.client.HTTPMessage at 0x7f4288fc6e30>)

In [10]:
import patoolib
patoolib.extract_archive("ILSVRC2012_img_val.tar", outdir="/content/ILSVRC2012_img_val")

INFO patool: Extracting ILSVRC2012_img_val.tar ...
INFO:patool:Extracting ILSVRC2012_img_val.tar ...
INFO patool: ... creating output directory `/content/ILSVRC2012_img_val'.
INFO:patool:... creating output directory `/content/ILSVRC2012_img_val'.
INFO patool: running /usr/bin/tar --extract --force-local --file ILSVRC2012_img_val.tar --directory /content/ILSVRC2012_img_val
INFO:patool:running /usr/bin/tar --extract --force-local --file ILSVRC2012_img_val.tar --directory /content/ILSVRC2012_img_val
INFO patool:     with input=''
INFO:patool:    with input=''
INFO patool: ... ILSVRC2012_img_val.tar extracted to `/content/ILSVRC2012_img_val'.
INFO:patool:... ILSVRC2012_img_val.tar extracted to `/content/ILSVRC2012_img_val'.


'/content/ILSVRC2012_img_val'

In [11]:
import os
import shutil
from tqdm import tqdm

# Paths
val_dir = '/content/ILSVRC2012_img_val'  # Directory containing the validation images
val_labels_file = '/content/ILSVRC2012_validation_ground_truth.txt'  # Validation labels file
output_dir = '/content/Complite_Dataset'  # Where you want to store organized images

# Load the class labels
with open(val_labels_file, 'r') as f:
    val_labels = f.readlines()

for i, label in tqdm(enumerate(val_labels), total= len(val_labels), desc="Prosesing Images"):
    label = label.strip()  # Remove newline characters
    img_name = f'ILSVRC2012_val_{i+1:08d}.JPEG'  # Image file naming format
    label_dir = os.path.join(output_dir, label)  # Directory for this label

    # Create label directory if it doesn't exist
    if not os.path.exists(label_dir):
        os.makedirs(label_dir)

    # Move image to corresponding directory
    img_path = os.path.join(val_dir, img_name)
    shutil.move(img_path, os.path.join(label_dir, img_name))

Prosesing Images: 100%|██████████| 50000/50000 [00:02<00:00, 24279.58it/s]


In [42]:
import os

# Path to the directory containing the folders
directory_path = '/content/Complite_Dataset'

# List all folders in the directory
folders = [folder for folder in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, folder))]

# Sort folders by their numeric names
sorted_folders = sorted(folders, key=lambda x: int(x))

# Print sorted folders
for folder in sorted_folders:
    print(folder)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [13]:
!pip install scipy

In [51]:
from scipy.io import loadmat
import pandas as pd
import csv

# Path to your .mat file
mat_file_path = '/content/meta.mat'

# Load the .mat file
mat_data = loadmat(mat_file_path)

# Prepare a list to store the labels
labels = []

# Loop over the first 1000 synsets and extract the labels
for i in range(1000):  # Limiting to the first 1000 entries
    label = mat_data['synsets'][i][0][1][0]  # Extract the label
    labels.append([label])  # Append the label as a list (for CSV format)

# Write the labels to a CSV file
with open('labels.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['labels'])  # Write the header
    writer.writerows(labels)  # Write the labels

print("CSV file 'labels.csv' created successfully.")

CSV file 'labels.csv' created successfully.


In [50]:
mat_data['synsets'][999]

array([(array([[1000]], dtype=uint16), array(['n03255030'], dtype='<U9'), array(['dumbbell'], dtype='<U8'), array(['an exercising weight; two spheres connected by a short bar that serves as a handle'],
             dtype='<U82'), array([[0]], dtype=uint8), array([], shape=(1, 0), dtype=uint8), array([[0]], dtype=uint8), array([[1300]], dtype=uint16))                                                   ],
      dtype=[('ILSVRC2012_ID', 'O'), ('WNID', 'O'), ('words', 'O'), ('gloss', 'O'), ('num_children', 'O'), ('children', 'O'), ('wordnet_height', 'O'), ('num_train_images', 'O')])

In [58]:
import pandas as pd
import os

# Path to the directory with folders
directory_path = '/content/Complite_Dataset'

# Path to the CSV file
csv_file_path = '/content/labels.csv'

# Read the CSV file using pandas
labels_df = pd.read_csv(csv_file_path)

# List all folders in the directory (assuming folders are named as numbers)
folders = [folder for folder in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, folder))]

# Sort folders numerically to match the order of labels in CSV
folders = sorted(folders, key=lambda x: int(x))

for i, folder in enumerate(folders):
    if i >= len(labels_df):
        print("Warning: More folders than labels in the CSV file.")
        break

    # Get the new name from the CSV
    new_name = labels_df['labels'][i]

    # Define the current folder path and the new folder path
    current_folder_path = os.path.join(directory_path, folder)
    new_folder_path = os.path.join(directory_path, new_name)

     # Rename the folder
    try:
        os.rename(current_folder_path, new_folder_path)
        print(f"Renamed folder '{folder}' to '{new_name}'")
    except Exception as e:
        print(f"Error renaming folder '{folder}' to '{new_name}': {e}")

Renamed folder '1' to 'n02119789'
Renamed folder '2' to 'n02100735'
Renamed folder '3' to 'n02110185'
Renamed folder '4' to 'n02096294'
Renamed folder '5' to 'n02102040'
Renamed folder '6' to 'n02066245'
Renamed folder '7' to 'n02509815'
Renamed folder '8' to 'n02124075'
Renamed folder '9' to 'n02417914'
Renamed folder '10' to 'n02123394'
Renamed folder '11' to 'n02125311'
Renamed folder '12' to 'n02423022'
Renamed folder '13' to 'n02346627'
Renamed folder '14' to 'n02077923'
Renamed folder '15' to 'n02110063'
Renamed folder '16' to 'n02447366'
Renamed folder '17' to 'n02109047'
Renamed folder '18' to 'n02089867'
Renamed folder '19' to 'n02102177'
Renamed folder '20' to 'n02091134'
Renamed folder '21' to 'n02092002'
Renamed folder '22' to 'n02071294'
Renamed folder '23' to 'n02442845'
Renamed folder '24' to 'n02504458'
Renamed folder '25' to 'n02092339'
Renamed folder '26' to 'n02098105'
Renamed folder '27' to 'n02096437'
Renamed folder '28' to 'n02114712'
Renamed folder '29' to 'n0210

In [59]:
dataset = load_dataset("imagefolder", data_dir="/content/Complite_Dataset")

Resolving data files:   0%|          | 0/50000 [00:00<?, ?it/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [60]:
dataset.push_to_hub("Tsomaros/Imagenet-1k_validation")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/15 [00:00<?, ?it/s]

Map:   0%|          | 0/3334 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/34 [00:00<?, ?ba/s]

Map:   0%|          | 0/3334 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/3333 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/34 [00:00<?, ?ba/s]

Map:   0%|          | 0/3333 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/34 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Tsomaros/Imagenet-1k_validation/commit/55405c49dece42420e68ddd5f80174f19b29ebaf', commit_message='Upload dataset', commit_description='', oid='55405c49dece42420e68ddd5f80174f19b29ebaf', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Tsomaros/Imagenet-1k_validation', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Tsomaros/Imagenet-1k_validation'), pr_revision=None, pr_num=None)